In [1]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

In [ ]:
#源代码
def get_data(code,start='2010-01-01',end='2020-07-16'):
    df=ts.get_k_data(code,autype='qfq',start=start,end=end)
    df.index=pd.to_datetime(df.date)
    df['openinterest']=0
    df=df[['open','high','low','close','volume','openinterest']]
    return df

In [2]:
datapath=r'/Users/nilei/Nutstore/Data/EURUSD_D1.csv'         #macos
dt=pd.read_csv(datapath)
dt.rename(columns={'Time (UTC)':'Date'},inplace=True)
dt.index=pd.to_datetime(dt.Date)
dt.head()

,Date,Open,High,Low,Close,Volume
Date,,,,,,
2010-01-01,2010.01.01 00:00:00,1.43283,1.43356,1.43181,1.43335,76787.4
2010-01-03,2010.01.03 00:00:00,1.43295,1.43359,1.42910,1.43141,6331.3
2010-01-04,2010.01.04 00:00:00,1.43143,1.44556,1.42559,1.44244,80019.4
2010-01-05,2010.01.05 00:00:00,1.44238,1.44834,1.43445,1.43634,79887.1
2010-01-06,2010.01.06 00:00:00,1.43638,1.44342,1.42807,1.44005,80971.8


In [3]:
dt.drop(columns='Date',inplace=True)
#数据太大，选择合适的时间段进行回测
dt_1=dt['2015-1-1':'2019-10-3']
dt_1.head()

,Open,High,Low,Close,Volume
Date,,,,,
2015-01-01,1.21038,1.21073,1.20783,1.20869,5732.39
2015-01-02,1.20871,1.20890,1.20004,1.20017,119384.46
2015-01-04,1.19495,1.19537,1.18677,1.19469,13740.64
2015-01-05,1.19469,1.19762,1.18872,1.19374,159296.56
2015-01-06,1.19374,1.19687,1.18461,1.18722,177669.80


In [6]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3285 entries, 2010-01-01 to 2020-07-01
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Open     3285 non-null   float64
 1   High     3285 non-null   float64
 2   Low      3285 non-null   float64
 3   Close    3285 non-null   float64
 4   Volume   3285 non-null   float64
dtypes: float64(5)
memory usage: 314.0 KB


In [ ]:
class TradeSizer(bt.Sizer):
    params = (('stake', 1),)    
    def _getsizing(self, comminfo, cash, data, isbuy):        
        if isbuy:          
            return self.p.stake        
        position = self.broker.getposition(data)        
        if not position.size:            
            return 0        
        else:            
            return position.size        
        return self.p.stake

In [ ]:
class TurtleStrategy(bt.Strategy):
    params = (('long_period',20),
              ('short_period',10),  
              ('printlog', False), )   

    def __init__(self):        

        self.order = None      
        self.buyprice = 0      
        self.buycomm = 0      
        self.buy_size = 0      
        self.buy_count = 0       
        # 海龟交易法则中的唐奇安通道和平均波幅ATR        
        self.H_line = bt.indicators.Highest(self.data.high(-1), period=self.p.long_period)        
        self.L_line = bt.indicators.Lowest(self.data.low(-1), period=self.p.short_period)       
        self.TR = bt.indicators.Max((self.data.high(0)- self.data.low(0)),\
                                    abs(self.data.close(-1)-self.data.high(0)), \
                                    abs(self.data.close(-1)  - self.data.low(0)))        
        self.ATR = bt.indicators.SimpleMovingAverage(self.TR, period=14)       
        # 价格与上下轨线的交叉      
        self.buy_signal = bt.ind.CrossOver(self.data.close(0), self.H_line)        
        self.sell_signal = bt.ind.CrossOver(self.data.close(0), self.L_line)    
    
    def next(self): 
        if self.order:
            return        
        #入场：价格突破上轨线且空仓时        
        if self.buy_signal > 0 and self.buy_count == 0:                                 
            self.buy_size = self.broker.getvalue() * 0.01 / self.ATR            
            self.buy_size  = int(self.buy_size  / 100) * 100                             
            self.sizer.p.stake = self.buy_size             
            self.buy_count = 1            
            self.order = self.buy()        
        #加仓：价格上涨了买入价的0.5的ATR且加仓次数少于3次（含）        
        elif self.data.close >self.buyprice+0.5*self.ATR[0] and self.buy_count > 0 and self.buy_count <=4:           
            self.buy_size  = self.broker.getvalue() * 0.01 / self.ATR            
            self.buy_size  = int(self.buy_size  / 100) * 100            
            self.sizer.p.stake = self.buy_size             
            self.order = self.buy()           
            self.buy_count += 1        
        #离场：价格跌破下轨线且持仓时        
        elif self.sell_signal < 0 and self.buy_count > 0:            
            self.order = self.sell()            
            self.buy_count = 0        
        #止损：价格跌破买入价的2个ATR且持仓时        
        elif self.data.close < (self.buyprice - 2*self.ATR[0]) and self.buy_count > 0:           
            self.order = self.sell()
            self.buy_count = 0   
            
    #交易记录日志（默认不打印结果）
    def log(self, txt, dt=None,doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()},{txt}')
    
    #记录交易执行情况（默认不输出结果）
    def notify_order(self, order):
        # 如果order为submitted/accepted,返回空
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 如果order为buy/sell executed,报告价格结果
        if order.status in [order.Completed]: 
            if order.isbuy():
                self.log(f'买入:\n价格:{order.executed.price},\
                成本:{order.executed.value},\
                手续费:{order.executed.comm}')
    
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f'卖出:\n价格：{order.executed.price},\
                成本: {order.executed.value},\
                手续费{order.executed.comm}')
            
            self.bar_executed = len(self) 
            
        # 如果指令取消/交易失败, 报告结果
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('交易失败')
        self.order = None
    
    #记录交易收益情况（可省略，默认不输出结果）
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        self.log(f'策略收益：\n毛收益 {trade.pnl:.2f}, 净收益 {trade.pnlcomm:.2f}')
    
    def stop(self):
        self.log(f'(组合线：{self.p.long_period},{self.p.short_period})； \
        期末总资金: {self.broker.getvalue():.2f}', doprint=True)

In [ ]:
def plot_stock(code,title,start,end):
    dd=ts.get_k_data(code,autype='qfq',start=start,end=end)
    dd.index=pd.to_datetime(dd.date)
    dd.close.plot(figsize=(14,6),color='r')
    plt.title(title+'价格走势\n'+start+':'+end,size=15)
    plt.annotate(f'期间累计涨幅:{(dd.close[-1]/dd.close[0]-1)*100:.2f}%', xy=(dd.index[-150],dd.close.mean()), 
             xytext=(dd.index[-500],dd.close.min()), bbox = dict(boxstyle = 'round,pad=0.5',
            fc = 'yellow', alpha = 0.5),
             arrowprops=dict(facecolor='green', shrink=0.05),fontsize=12)
    plt.show()

In [ ]:
def main(code,long_list,short_list,start,end='',startcash=1000000,com=0.001):
    #创建主控制器
    cerebro = bt.Cerebro()      
    #导入策略参数寻优
    cerebro.optstrategy(TurtleStrategy,long_period=long_list,short_period=short_list)    
    #获取数据
    df=ts.get_k_data(code,autype='qfq',start=start,end=end)
    df.index=pd.to_datetime(df.date)
    df=df[['open','high','low','close','volume']]
    #将数据加载至回测系统
    data = bt.feeds.PandasData(dataname=df)    
    cerebro.adddata(data)
    #broker设置资金、手续费
    cerebro.broker.setcash(startcash)           
    cerebro.broker.setcommission(commission=com)    
    #设置买入设置，策略，数量
    cerebro.addsizer(TradeSizer)    
    print('期初总资金: %.2f' % cerebro.broker.getvalue())    
    cerebro.run(maxcpus=1)    

In [ ]:
plot_stock('sh','上证综指','2010-01-01','2020-07-17')

In [ ]:
long_list=range(20,70,5)
short_list=range(5,20,5)
main('sh',long_list,short_list,'2010-01-01','2020-07-17')

In [ ]:
def performance(code,long,short,start,end,startcash,com):
    cerebro = bt.Cerebro()      
    #导入策略参数寻优
    cerebro.addstrategy(TurtleStrategy,long_period=long,short_period=short)    
    #获取数据
    df=ts.get_k_data(code,autype='qfq',start=start,end=end)
    df.index=pd.to_datetime(df.date)
    df=df[['open','high','low','close','volume']]
    #将数据加载至回测系统
    data = bt.feeds.PandasData(dataname=df)    
    cerebro.adddata(data)
    #broker设置资金、手续费
    cerebro.broker.setcash(startcash)           
    cerebro.broker.setcommission(commission=com)    
    #设置买入设置，策略，数量
    cerebro.addsizer(TradeSizer)   
    df00,df0,df1,df2,df3,df4=bt.out_result(cerebro)
    return df00,df0,df1,df2,df3,df4

In [ ]:
df00,df0,df1,df2,df3,df4=performance('sh',50,5,'2010-01-01','2020-07-17',1000000,0.001)

In [ ]:
#df00

In [ ]:
from pyecharts import Line,Bar
def plot_result_py(data,v,title,plot_type='line',zoom=False):
    att=data.index
    try:
        attr=att.strftime('%Y%m%d')
    except:
        attr=att
    if plot_type=='line':
        p=Line(title)
        p.add('',attr,list(data[v].round(2)),
         is_symbol_show=False,line_width=2,
        is_datazoom_show=zoom,is_splitline_show=True)
    else:
        p=Bar(title)
        p.add('',attr,[int(i*1000)/10 for i in list(data[v])],
              is_label_show=True,
        is_datazoom_show=zoom,is_splitline_show=True)
    return p

In [ ]:
#plot_result_py(df0,'total_value','账户价值')

In [ ]:
#plot_result_py(df4,'total_position_value','持仓市值')

In [ ]:
#plot_result_py(df3,'year_rate','年化收益%',plot_type='bar')

In [ ]:
#plot_stock('601318','中国平安','2010-01-01','2020-07-17')

In [ ]:
#long_list=range(20,70,5)
#short_list=range(5,20,5)
#main('601318',long_list,short_list,'2010-01-01','2020-07-16',startcash=100000)

In [ ]:
zgpa00,zgpa0,zgpa1,zgpa2,zgpa3,zgpa4=performance('601318',20,10,'2010-01-01','2020-07-17',100000,0.001)

In [ ]:
#zgpa00

In [ ]:
#plot_result_py(zgpa0,'total_value','账户价值')

In [ ]:
#plot_result_py(zgpa4,'total_position_value','持仓市值')

In [ ]:
#plot_result_py(zgpa3,'year_rate','年化收益%',plot_type='bar')